In [2]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Apathy

In [3]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\Apathy
data = pd.read_csv('ApathyCateg_Methylome_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\Apathy


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,MDS_Apathy_Category,cg19958696,cg05962325,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.014601,0.068728,...,0.630319,0.85156,0.52162,0.495925,0.830013,0.076462,0.72859,0.246918,0.039718,0.024071


In [4]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','APPRDX','MDS_Apathy_Category'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg19958696  cg05962325  cg15093662  cg04873221  cg05903298  cg19020436  \
0    0.014601    0.068728    0.019329    0.021747    0.012923    0.017399   
1    0.013334    0.038642    0.017604    0.021562    0.012244    0.016683   

   cg14994056  cg05973813  cg18337422  cg11738745  ...  cg19795267  \
0    0.033939    0.020282    0.017160    0.012127  ...    0.630319   
1    0.036065    0.021018    0.013908    0.011829  ...    0.655485   

   cg16922840  cg19628277  cg26480988  cg01673347  cg12994818  cg09008531  \
0     0.85156    0.521620    0.495925    0.830013    0.076462     0.72859   
1     0.86754    0.546537    0.511772    0.802469    0.033789     0.63495   

   cg16463148  cg04021430  cg04863888  
0    0.246918    0.039718    0.024071  
1    0.193630    0.023717    0.023474  

[2 rows x 3564 columns]


In [5]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [6]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6612903225806451

In [7]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=24,
                                          random_state=RandomState(MT19937) at 0x1F3F5678340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F3F5678340, verbose=1)

In [8]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [9]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [10]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
3506,cg13959611,1
1807,cg02157463,1
2633,cg19125584,2


In [13]:
Top50.to_csv ('Apathy_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('Apathy_Boruta_All_Male.csv', index = False)

In [12]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Apathy_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11424\1187690991.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg07584368,cg02699673,cg10957063,cg08955609,cg10400309,cg03984780,cg11865360,cg08409074,cg08702946,cg25407140,...,cg25361663,cg25288762,cg23362473,cg16643803,cg01447828,cg26126367,cg07897734,cg06552618,cg13959611,APPRDX
0,0.868743,0.621165,0.630408,0.046059,0.013044,0.065929,0.279839,0.036602,0.922071,0.116656,...,0.911043,0.85890,0.917361,0.029913,0.441146,0.825690,0.842938,0.035788,0.044262,1
1,0.864397,0.576635,0.632613,0.026931,0.010265,0.066287,0.098743,0.020570,0.922684,0.077034,...,0.929506,0.85435,0.900393,0.033853,0.559728,0.852642,0.894486,0.031247,0.048167,2


# SCOPA Sex

In [14]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\ScopaSex
data = pd.read_csv('SexMaleCateg_Methylome_APPRDX.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\ScopaSex


,Sentrix,PATNO,HYS,Gender,SCOPA_Sex,APPRDX,cg17370322,cg15396799,cg15388598,cg14189621,...,cg00600684,cg13207036,cg05750276,cg02312170,cg17391830,cg13151449,cg13596132,cg26538046,cg18607338,cg14149699
0,200991620021_R01C01,3001,2,1,2,1,0.013535,0.242275,0.01208,0.024553,...,0.927369,0.0306,0.052903,0.053346,0.041092,0.902988,0.911403,0.033966,0.827098,0.030215


In [15]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','SCOPA_Sex','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg17370322  cg15396799  cg15388598  cg14189621  cg15158045  cg09770095  \
0    0.013535    0.242275    0.012080    0.024553    0.929080    0.953919   
1    0.012805    0.109881    0.018853    0.052400    0.945216    0.927368   

   cg05412028  cg26685941  cg25868675  cg24107165  ...  cg00600684  \
0    0.091946    0.082563    0.015361    0.686841  ...    0.927369   
1    0.117943    0.098877    0.019414    0.563660  ...    0.920645   

   cg13207036  cg05750276  cg02312170  cg17391830  cg13151449  cg13596132  \
0    0.030600    0.052903    0.053346    0.041092    0.902988    0.911403   
1    0.023915    0.023711    0.035630    0.027145    0.896837    0.889727   

   cg26538046  cg18607338  cg14149699  
0    0.033966    0.827098    0.030215  
1    0.038348    0.836846    0.083898  

[2 rows x 10534 columns]


In [16]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [17]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6774193548387096

In [18]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100


BorutaPy finished running.

Iteration: 	30 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	10533


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1F3F8834140),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F3F8834140, verbose=1)

In [19]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [20]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [21]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
1519,cg05817202,1
3007,cg03682895,2
7783,cg19380722,3


In [22]:
Top50.to_csv ('ScopaSex_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('ScopaSex_Boruta_All_Male.csv', index = False)

In [23]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['PATNO'] = data['PATNO']
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('ScopaSex_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11424\2492820744.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['PATNO'] = data['PATNO']
C:\Users\system 4\AppData\Local\Temp\ipykernel_11424\2492820744.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg06751221,cg22870386,cg07588779,cg20095680,cg22807314,cg11865360,cg05817202,cg19612504,cg26243905,cg17422176,...,cg02477305,cg07659840,cg14595995,cg24272338,cg09119776,cg02153334,cg03303515,cg24120153,PATNO,APPRDX
0,0.057391,0.828698,0.082613,0.035714,0.920441,0.279839,0.823899,0.954524,0.837745,0.117596,...,0.916772,0.833857,0.888622,0.912405,0.524141,0.942443,0.245769,0.713483,3001,1
1,0.083624,0.888004,0.095515,0.046304,0.926729,0.137535,0.849287,0.965545,0.900153,0.147173,...,0.923712,0.880487,0.915410,0.891393,0.551746,0.898963,0.211177,0.783492,3007,1
